In [1]:
from tqdm import tqdm
from scipy import io
from scipy import sparse
import scipy
import gzip
import scanpy

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import DataLoader, Dataset
from torch.distributions.beta import Beta

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import os
import pickle

In [2]:
# 0 A Census of Immune Cells
# ~ 71 min
DIR_PATH = "/data/home/kimds/Data/A Census of Immune Cells"
census_blood = scanpy.read_loom(DIR_PATH + "/1M-immune-human-blood-10XV2.loom")
census_immune = scanpy.read_loom(DIR_PATH + "/1M-immune-human-immune-10XV2.loom")
census_of_immune_cells_genes = pd.read_csv(DIR_PATH+'/genes.csv')

census_blood.obs_names_make_unique()
census_blood.var_names_make_unique()
census_immune.obs_names_make_unique()
census_immune.var_names_make_unique()

census_raw = sparse.vstack([census_blood.X, census_immune.X]).T

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
/data/home/kimds/anaconda3/envs/TORCH/lib/python3.8/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/data/home/kimds/anaconda3/envs/TORCH/lib/python3.8/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/data/home/kimds/anaconda3/envs/TORCH/lib/python3.8/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/data/home/kimds/anaconda3/envs/TORCH/lib/python3.8/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_n

In [9]:
# 3 Heart Cell Atlas
# ~ 1 min
DIR_PATH = '/data/home/kimds/Data/Heart Cell Atlas'
heart_raw = io.mmread(DIR_PATH+'/'+'sparse_mtx.mtx')
heart_genes = pd.read_csv(DIR_PATH+'/genes.csv')

In [4]:
# 4 Immune Cell Atlas
# ~ 16 min
DIR_PATH = '/data/home/kimds/Data/Immune Cell Atlas'
immune_raw = io.mmread(DIR_PATH+'/'+'sparse_mtx.mtx')
immune_genes = pd.read_csv(DIR_PATH+'/genes.csv')

In [5]:
# 5 Immune Cells in Critical COVID19
# ~ 60 min
DIR_PATH = '/data/home/kimds/Data/Immune Cells in Critical COVID19'
FILE_PATH = '/data/home/kimds/Data/Immune Cells in Critical COVID19/GSE158055_covid19_counts.mtx.gz'
covid_raw = io.mmread(FILE_PATH)
covid_features = pd.read_csv(DIR_PATH+'/GSE158055_covid19_features.tsv.gz', sep='\t', compression='gzip', header=None)


In [101]:
census_genes = census_blood.var.index

In [5]:
DIR_PATH = '/data/home/kimds/Data/Heart Cell Atlas'
heart_genes = pd.read_csv(DIR_PATH+'/genes.csv')
heart_genes = heart_genes.iloc[:, 0]

In [6]:
DIR_PATH = '/data/home/kimds/Data/Immune Cell Atlas'
immune_genes = pd.read_csv(DIR_PATH+'/genes.csv')
immune_genes = immune_genes.iloc[:, 0]

In [7]:
DIR_PATH = '/data/home/kimds/Data/Immune Cells in Critical COVID19'
covid_features = pd.read_csv(DIR_PATH+'/GSE158055_covid19_features.tsv.gz', sep='\t', compression='gzip', header=None)
covid_genes = covid_features.iloc[:, 0]

In [102]:
temp = census_genes
temp = temp[temp.isin(heart_genes)]
temp = temp[temp.isin(immune_genes)]
temp = temp[temp.isin(covid_genes)]
census_indices = census_genes.isin(temp)
heart_indices = heart_genes.isin(temp)
immune_indices = immune_genes.isin(temp)
covid_indices = covid_genes.isin(temp)

In [105]:
census = census_raw.tocsc()[census_indices, :]
heart = heart_raw.tocsc()[heart_indices, :]
immune = immune_raw.tocsc()[immune_indices, :]
covid = covid_raw.tocsc()[covid_indices, :]


In [106]:
data = sparse.hstack([census, heart, immune, covid])